# Numpy Distance versus Tensorflow 

In [ ]:
from MDAnalysis.core.distances import distance_array, self_distance_array
from MDAnalysis.analysis.distances import contact_matrix
import scipy.sparse
from scipy.spatial.distance import cdist
import numpy as np
import time, os, sys, gc
import datetime
import logging
import tensorflow as tf
logger = logging.getLogger("py4j")
logger.setLevel(logging.ERROR)
import warnings
warnings.filterwarnings("ignore")

# Small Test Data

In [ ]:
x = np.array((np.random.rand(10, 3)*10).astype(int))

# Numpy Broadcast Version

We extend both our 2-D array into 3 dimensions so that we can do an element-wise subtraction (broadcast): `x[:, None, :]`


From **Python Data Science Handbook** by Jake Vanderblas:

Broadcasting in NumPy follows a strict set of rules to determine the interaction between the two arrays:

1. If the two arrays differ in their number of dimensions, the shape of the one with fewer dimensions is padded with ones on its leading (left) side.

1. If the shape of the two arrays does not match in any dimension, the array with shape equal to 1 in that dimension is stretched to match the other shape.

1. If in any dimension the sizes disagree and neither is equal to 1, an error is raised.



In [ ]:
x[:, None, :].shape

In [ ]:
x.shape

In [ ]:
(x[:, None, :]-x).shape

In [ ]:
x.reshape(3,10)

In [ ]:
def dist(x): 
    return np.sqrt(((x[:, None, :] - x) ** 2).sum(-1))

In [ ]:
dist(x)

# Scikit Learn

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances
euclidean_distances(x,x)

In [ ]:
from scipy.spatial.distance import pdist
pdist(x)

# MD Analysis

In [ ]:
def compute_distance_mdanalysis(coord):
    start = time.time()
    contact_matrix(coord, returntype="sparse")
    result="ComputeDistanceMDAnalysisSparse, %d, %.2f"%(len(coord), (time.time()-start))
    return result

# Tensorflow

In [ ]:
import tensorflow as tf
import numpy as np

def compute_distance_tf(coord):
    start = time.time()
    sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
    matrix1 = tf.convert_to_tensor(coord.astype("float32"))
    matrix_extend=tf.expand_dims(matrix1, 1)
    dist_matrix=tf.sub(matrix_extend, matrix1)
    dist_matrix_pow=tf.pow(dist_matrix, 2)
    dist_matrix_pow_red=tf.reduce_sum(dist_matrix_pow, 2)
    dist_matrix_euc=tf.sqrt(dist_matrix_pow_red)
    rc=sess.run(dist_matrix_euc)
    result="ComputeDistanceTensorflow, %d, %.2f"%(len(coord), (time.time()-start))
    sess.close()
    return result, rc

In [ ]:
def compute_distance_tf2(coord):
    start = time.time()
    sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
    matrix1 = tf.convert_to_tensor(coord.astype("float32"))
    expanded_vectors = tf.expand_dims(matrix1, 0)
    expanded_centroids = tf.expand_dims(matrix1, 1)
    distances = tf.sqrt(tf.reduce_sum(tf.square(tf.sub(expanded_vectors, expanded_centroids)), 2))
    rc=sess.run(distances)
    result="ComputeDistanceTensorflow2, %d, %.2f"%(len(coord), (time.time()-start))
    sess.close()
    #print rc
    return rc

In [ ]:
result= compute_distance_tf2(x)
result

In [ ]:
result

http://esciencegroup.com/2016/01/05/an-encounter-with-googles-tensorflow/
http://learningtensorflow.com/lesson6/

In [ ]:
cutoff = 15.0

def get_edges_point(self, point_index, adjacency_matrix, cutoff=15.0):
    edge_list = []
    for index, i in np.ndenumerate(adjacency_matrix):
        print ("Index: %d, Value: %d"%(index[i], i))
        #if i==True and point_index<=index[1]:
        if point_index<=index[1] and i<cutoff:
            # Attention we only compute the upper half of the adjacency matrix
            # thus we need to offset the target edge vertice by point_index
            edge_list.append((point_index, point_index+index[1]))
    return edge_list
    
def compute_distance_tf_batch(coord, batch_size=5):
    start_time = time.time()
    num_batches = len(coord)/batch_size
    sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
    matrix2 = tf.convert_to_tensor(coord.astype("float32"))
    print matrix2.get_shape()
    matrix1_splits = tf.split(0, num_batches, matrix2)
    edges=[]
    for split in matrix1_splits:
    #for i in range(num_batches):
        #start=i*batch_size
        #end=start+batch_size
        #print "Compute batch from %d to %d"%(start, end)
        #matrix1 = tf.convert_to_tensor(coord[start:end].astype("float32"))
        #matrix1 = tf.slice(matrix2, [start], [batch_size])
        matrix1 = split
        matrix_extend=tf.expand_dims(matrix1, 1)
        dist_matrix=tf.sub(matrix_extend, matrix2)
        dist_matrix_pow=tf.pow(dist_matrix, 2)
        dist_matrix_pow_red=tf.reduce_sum(dist_matrix_pow, 2)
        dist_matrix_euc=tf.sqrt(dist_matrix_pow_red)>cutoff
        dist_custoff_true=tf.where(dist_matrix_euc)
        #edges.append(sess.run(dist_matrix_euc))
        edges.append(sess.run(dist_custoff_true))
    sess.close()    
    result="ComputeDistanceTensorflow, %d, %.2f"%(len(coord), (time.time()-start_time))
    return (result, edges)

In [ ]:
len(x)/3

In [ ]:
compute_distance_tf_batch(x)

# Benchmark

In [ ]:
DATA_PATH="../../data/mdanalysis/synthetic/traj/"
files=[os.path.join(DATA_PATH, i) for i in os.listdir(DATA_PATH)]

files=["../../data/mdanalysis/synthetic/traj/10000.np_txt"]

for file_name in files:
    print "Process: " + file_name
    coord = np.loadtxt(file_name, dtype='float32')
    start = time.time()
    result=compute_distance_tf_batch(coord, batch_size=4000)
    print result[0]

In [ ]:
!ls "../../data/mdanalysis/synthetic/traj/"

# Theano

In [ ]:
import theano
import theano.tensor as T
from theano import shared

tx=shared(x)
diff=tx-tx
diff_squared=diff**2

diff_squared.eval()